In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Day 4 — Mini Automation: Clean & Export Reports\n",
    "\n",
    "This notebook demonstrates a small, production-friendly cleaning pipeline using **pandas**, creates export-ready reports (CSV), and produces a quick verification visualization. Use this as a template to apply on real CSVs.\n",
    "\n",
    "Files this notebook will create (in `reports/day4/`):\n",
    "- `sales_by_region.csv`\n",
    "- `top_customers_by_sales.csv`\n",
    "- `monthly_summary.csv`\n",
    "\n",
    "Run each cell sequentially."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Imports\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "from pathlib import Path\n",
    "import matplotlib.pyplot as plt\n",
    "%matplotlib inline\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1) Create a messy in-memory dataset (or replace with `pd.read_csv()` to load a file)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "raw = {\n",
    "    \"order_id\": [1011, 1021, 1033, 1054, 1066, 1077, 1088],\n",
    "    \"customer_id\": [11, 21, 33, 54, 11, 21, 33],\n",
    "    \"region\": [\" Pune \", \"Mumbai\", \"nagpur\", \"Nashik\", None, \"Mumbai\", \"Nagpur\"],\n",
    "    \"amount\": [\"25,000\", \"15000\", \"28,000\", \"52000\", \"12,000\", None, \" 30,000 \"],\n",
    "    \"order_date\": [\"2025-03-11\", \"2025/05/03\", \"2025-04-01\", \"2025-08-12\", \"2025-09-10\", \"2025-09-15\", \"2025-09-20\"]\n",
    "}\n",
    "df_raw = pd.DataFrame(raw)\n",
    "df_raw.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2) Cleaning pipeline (function) — modular and reusable"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def clean_sales_df(df: pd.DataFrame) -> pd.DataFrame:\n",
    "    df = df.copy()\n",
    "    # Standardize whitespace and case\n",
    "    df['region'] = df['region'].astype(str).str.strip().replace({'None': None})\n",
    "    df['region'] = df['region'].replace('nan', None)\n",
    "    # Clean amount: remove commas/spaces and coerce\n",
    "    df['amount'] = df['amount'].astype(str).str.replace(',', '').str.strip()\n",
    "    df['amount'] = pd.to_numeric(df['amount'], errors='coerce')\n",
    "    # Parse dates\n",
    "    df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')\n",
    "    # Fill missing values sensibly\n",
    "    df['region'] = df['region'].fillna('Unknown')\n",
    "    df['amount'] = df['amount'].fillna(0).astype(int)\n",
    "    # Drop rows with invalid dates (or optionally handle them)\n",
    "    df = df.dropna(subset=['order_date'])\n",
    "    df = df[['order_id', 'customer_id', 'region', 'amount', 'order_date']]\n",
    "    return df\n",
    "\n",
    "# Run cleaning\n",
    "df = clean_sales_df(df_raw)\n",
    "df"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3) Create exportable reports"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def create_reports(df: pd.DataFrame, out_dir: Path):\n",
    "    out_dir.mkdir(parents=True, exist_ok=True)\n",
    "    # Sales by region\n",
    "    sales_region = df.groupby('region', as_index=False)['amount'].sum().rename(columns={'amount': 'total_sales'})\n",
    "    sales_region = sales_region.sort_values('total_sales', ascending=False)\n",
    "    sales_region.to_csv(out_dir / \"sales_by_region.csv\", index=False)\n",
    "    # Top customers\n",
    "    top_customers = df.groupby('customer_id', as_index=False)['amount'].sum().rename(columns={'amount': 'total_spent'})\n",
    "    top_customers = top_customers.sort_values('total_spent', ascending=False)\n",
    "    top_customers.to_csv(out_dir / \"top_customers_by_sales.csv\", index=False)\n",
    "    # Monthly summary\n",
    "    df['order_month'] = df['order_date'].dt.to_period('M')\n",
    "    monthly = df.groupby('order_month')['amount'].agg(['sum','mean','count']).reset_index()\n",
    "    monthly.to_csv(out_dir / \"monthly_summary.csv\", index=False)\n",
    "    return sales_region, top_customers, monthly\n",
    "\n",
    "out_dir = Path('reports/day4')\n",
    "sales_region, top_customers, monthly = create_reports(df, out_dir)\n",
    "print('Reports saved to', out_dir.resolve())\n",
    "sales_region, top_customers.head(), monthly"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4) Quick visualization to validate output"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "plt.figure(figsize=(7,4))\n",
    "plt.bar(sales_region['region'], sales_region['total_sales'])\n",
    "plt.title('Sales by Region')\n",
    "plt.ylabel('Total Sales (₹)')\n",
    "plt.xlabel('Region')\n",
    "plt.xticks(rotation=30)\n",
    "plt.grid(axis='y', linestyle='--', alpha=0.4)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5) Optional: package into a small CLI function (example)\n",
    "\n",
    "You can wrap this pipeline in a function to run nightly (cron/Task Scheduler) or trigger from a small command-line interface."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def run_pipeline(source_df=None, save_reports=True, out_dir=Path('reports/day4')):\n",
    "    if source_df is None:\n",
    "        source_df = df_raw\n",
    "    cleaned = clean_sales_df(source_df)\n",
    "    sales_region, top_customers, monthly = create_reports(cleaned, Path(out_dir))\n",
    "    return {'cleaned': cleaned, 'sales_region': sales_region, 'top_customers': top_customers, 'monthly': monthly}\n",
    "\n",
    "# Example run\n",
    "res = run_pipeline()\n",
    "res['sales_region']"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Save and commit\n",
    "\n",
    "Files generated by this notebook are in `reports/day4/`. Add them to your repo if you'd like to store the generated CSVs as artifacts."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}


NameError: name 'null' is not defined